In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
# set the tracking uri
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
# Load the dataset
X, y = datasets.load_iris(return_X_y=True)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Define the model hyperparameter
params = {"penalty": "l2", "solver" : "lbfgs", "max_iter" : 1000, "multi_class" : "auto", "random_state" : 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

/Users/kyuho/Codes/learn-mlflow/MlFlowStarter/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [4]:
# Prediction on the test data
y_pred=lr.predict(X_test)
y_pred

array([1, 1, 2, 1, 0, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 1,
       2, 2, 1, 1, 1, 1, 1, 0])

In [5]:
acc = accuracy_score(y_test, y_pred)
acc

0.9666666666666667

In [6]:
# MLFlow tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# create a new MLflow experiment
mlflow.set_experiment("MLflow Quick Start")

# Start an MLflow run
with mlflow.start_run():
    # log hyperparameters
    mlflow.log_params(params)

    # lg the accuracy metrics
    mlflow.log_metric("accuracy", acc)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name = "tracking-quickstart"
    )

2025/01/08 09:31:09 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quick Start' does not exist. Creating a new experiment.
Successfully registered model 'tracking-quickstart'.
2025/01/08 09:31:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run upbeat-hog-108 at: http://127.0.0.1:5000/#/experiments/778183649407657242/runs/831d7a87c73f4f38a02b872da17376fd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/778183649407657242


Created version '1' of model 'tracking-quickstart'.


In [7]:
params = {"solver" : "newton-cg", "max_iter" : 1000, "multi_class" : "auto", "random_state" : 1000}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

/Users/kyuho/Codes/learn-mlflow/MlFlowStarter/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [8]:
# Prediction on the test data
y_pred=lr.predict(X_test)
y_pred

array([1, 1, 2, 1, 0, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 1,
       2, 2, 1, 1, 1, 1, 1, 0])

In [9]:
acc = accuracy_score(y_test, y_pred)
acc

0.9666666666666667

In [10]:
# Start an MLflow run
with mlflow.start_run():
    # log hyperparameters
    mlflow.log_params(params)

    # lg the accuracy metrics
    mlflow.log_metric("accuracy", acc)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name = "tracking-quickstart"
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/01/08 09:31:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2


🏃 View run unique-bass-232 at: http://127.0.0.1:5000/#/experiments/778183649407657242/runs/685880180c474fb7a7e3eb73189f9975
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/778183649407657242


Created version '2' of model 'tracking-quickstart'.


In [11]:
model_info.model_uri

'runs:/685880180c474fb7a7e3eb73189f9975/iris_model'

## Inferencing And Validating  Model

In [17]:
from mlflow.models import validate_serving_input

model_uri = model_info.model_uri

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    
    ...

    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      6.6,
      2.9,
      4.6,
      1.3
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([0, 1, 0, 0, 2, 0, 2, 1, 0, 1, 2, 2, 1, 1, 1, 0, 2, 2, 1, 1, 1, 2,
       2, 2, 1, 2, 2, 0, 2, 1, 1, 0, 2, 1, 0, 1, 1, 0, 0, 2, 1, 0, 1, 0,
       2, 1, 0, 2, 0, 1, 2, 0, 0, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 2, 1, 1,
       2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 2, 2, 0, 2, 0, 1, 0, 0, 2, 1, 1, 1,
       2, 0, 2, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 1, 1, 0, 1, 0, 0, 0, 2, 2,
       2, 1, 0, 0, 1, 0, 1, 1, 2, 1])

## Load the model back for prediction as a generic python function model

In [19]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_features_name = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_features_name)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result

mlflow.pyfunc.loaded_model:
  artifact_path: iris_model
  flavor: mlflow.sklearn
  run_id: d284f9dd6d4249bba3281c672bc285e3



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,5.2,2.7,3.9,1.4,1,1
1,6.7,3.3,5.7,2.1,2,2
2,6.9,3.1,4.9,1.5,1,1
3,6.5,3.0,5.8,2.2,2,2
4,4.3,3.0,1.1,0.1,0,0
5,4.6,3.2,1.4,0.2,0,0
6,5.5,4.2,1.4,0.2,0,0
7,5.4,3.4,1.5,0.4,0,0
8,5.1,3.5,1.4,0.2,0,0
9,4.9,3.0,1.4,0.2,0,0


## Model Registry

In [20]:
# Start an MLflow run
with mlflow.start_run():
    # log hyperparameters
    mlflow.log_params(params)

    # lg the accuracy metrics
    mlflow.log_metric("accuracy", 1.0)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info2", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        # registered_model_name = "tracking-quickstart"
    )

🏃 View run awesome-tern-843 at: http://127.0.0.1:5000/#/experiments/875863669709788086/runs/512bd8f87d18403ab8eac190ae4e4453
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/875863669709788086


## Inferencing from model from model registry

In [22]:
import mlflow.sklearn

model_name = "tracking-quickstart"
model_version = "4"

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)

model

LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [23]:
y_pred_new = model.predict(X_test)

y_pred_new

array([1, 2, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 2, 1, 1, 1, 2,
       1, 1, 0, 1, 0, 2, 2, 2])